History

* 2020.06.04 Fix print bug if find only one data .  
* 2020.06.05 Add tag_type `基隆市文化局` .
* 2020.06.06 Add price unit .
* 2020.11.25 upload definition(`price_str`) for no price tender .
 - fix bug for reflash data .
 - upload Bot_Message for too many tender .
* 2021.01.14 Add holiday parse and set time in 09:00 .
* 2021.01.17 set more checktime at 15:00 .
 - push to git .
* 2021.01.18 set time countdown module .  
* 2021.05.19 Use docker run .

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import json
import os

with open('Secret_file.json',"r") as f:
    tender_token = json.load(f)['Slack_API']['Tender']

FileNotFoundError: [Errno 2] No such file or directory: 'Secret_file.json'

In [2]:
# Tag設定

tags = ['桌遊','教材', '教具', '桌上遊戲', '桌上型遊戲', '遊戲', '兒童', '益智', '贈品', '禮品', '探索', '紙牌', '沉浸式體驗', '玩具', '解謎', '實境解謎', '文化轉譯']
organ_tags = ['基隆市文化局']

In [3]:
# 2021月份工作日設定


# 周一到週五放假的日期
HOLIDAY = ["2021-02-10", "2021-02-11", "2021-02-12", "2021-02-15", "2021-02-16",
           "2021-03-01",
           "2021-04-02", "2021-04-05", "2021-04-30",
           "2021-06-14",
           "2021-09-20", "2021-09-21",
           "2021-10-11",
           "2021-12-31"]
# 週六週日還要上班的日期
WORKDAY = ["2021-02-20", "2021-09-11"]

In [4]:
# 重啟公告用
def Bot_Message(channel, text):
    payload={
        "token":tender_token,
        "channel":channel,
        "text":'{}'.format(text)} 
    r = requests.post("https://slack.com/api/chat.postMessage", params=payload) 

Bot_Message('tenderlist', '標案通知已重新啟動\n每日09:00與15:00通知新標案\n有任何需求請<@D.E.>協助')
Bot_Message('tenderlist', '關鍵字： '+' / '.join(tags) + '\n關注機關： '+organ_tags[0])

In [4]:
def price_str(price):
    len_price_x, len_price_y = len(price)//3, len(price)%3
    list_price = list(price)
    if len_price_y > 0:
        for i in range(1, len_price_x+1):
            list_price.insert(-3*i+(-i+1), ',')
        price = ''.join(list_price)
    elif len_price_y ==0 and len_price_x >0:
        for i in range(1, len_price_x):
            list_price.insert(-3*i+(-i+1), ',')
        price = ''.join(list_price)
    else:
        price = 'no price data .'
    return price

def base_file():
    file = {'method': 'search',
            'searchMethod': 'true',
            'hid_1': 1,
            'tenderType': 'tenderDeclaration',
            'tenderWay': '1,2,3,4,5,6,7,10,12',
            'tenderDateRadio': 'on',
            'isSpdt': 'Y',
            'btnQuery': '查詢'}
    return file

def get_list(item, tag):
    ss = item.find_all('td', attrs={'align':'left'})
    temp = []
    temp.append(tag) #tag
    temp.append(ss[0].text.rstrip('\xa0')) #name
    temp.append(ss[1].text.split('\n')[1].strip('\t')) #ID
    if ss[1].text.split('\n')[3].strip('\t') != '(更正公告)':
        temp.append(ss[1].text.split('\n')[8].strip('\t')) #describe
    else:
        temp.append(ss[1].text.split('\n')[10].strip('\t')) #describe
    temp.append(ss[2].text) #how
    temp.append(price_str(item.find('td', attrs={'align':'right'}).text.replace('\t', '').replace('\n', ''))) #price
    temp.append(ss[3].text) #type
    temp.append(ss[4].text.rstrip('\n')) #start_time
    temp.append(ss[5].text.rstrip('\n')) #deadline
    temp.append('http://web.pcc.gov.tw/tps'+(ss[1].find('a')['href']).lstrip('..')) #url
    return temp

In [5]:
# 首次資料庫建立與通知
url = 'http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic'
item_list = []
'''
normal_tag
'''
file = base_file()
for tag in tags:
    file['tenderName'] = tag
    r = requests.post(url, data=file)
    soup = BeautifulSoup(r.text, 'html5lib')

    try:
        items = soup.find_all('tr', attrs={'onmouseover':'overcss(this);'})
    except:
        time.sleep(3)
        continue

    for item in items:
        temp = get_list(item, tag)
        item_list.append(temp)
    time.sleep(5)

'''
organ_tag
'''
file = base_file()
for organ_tag in organ_tags:
    file['orgName'] = organ_tag
    r = requests.post(url, data=file)
    soup = BeautifulSoup(r.text, 'html5lib')

    try:
        items = soup.find_all('tr', attrs={'onmouseover':'overcss(this);'})
    except:
#                 time.sleep(3) #目前只有一例故取消延遲
        continue

    for item in items:
        temp = get_list(item, organ_tag)
        item_list.append(temp)
#             time.sleep(5) #目前只有一例故取消延遲

df = pd.DataFrame(item_list, columns=['tag', '機關名稱', '標案案號', '標案名稱', '招標方式', '預算金額', '採購性質', '公告日期', '截止日期', 'url'])
df.drop_duplicates('標案案號', inplace=True)
df.to_csv('/root/tenderdata/tender_list.csv', index=False)
Bot_Message('tenderlist', '資料庫重新建立：{}\n再麻煩確認這個時間點之前的標案是否遺漏。'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
# docker裡面是UTC時間，下次修正再調整UTC+8

In [ ]:
'''
時間阻斷器
'''
# 暫時只用小時設定
def time_start(hour):
    # 時
    while True:
        if datetime.now().hour == (hour-1):
            break
        else:
            time.sleep(3600)
            continue

    # 分
    while True:
        if datetime.now().minute == 59:
            break
        else:
            time.sleep(60)
            continue

    # 秒
    while True:
        if datetime.now().second >= 50:
            break
        else:
            time.sleep(5)
            continue

    while True:
        if datetime.now().hour == hour:
            break
        else:
            continue

if datetime.now().hour<9 or datetime.now().hour>=15:
    time_start(9)
else:
    time_start(15)
print('執行時間：\n', datetime.now())

----

In [ ]:
if __name__=='__main__':
    url = 'http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic'
    check_point = 0
    while check_point <5:
        try:
            start_time = time.time() #1 【計時器】
            today = datetime.now()

            # 工作判定器
            if (today.weekday()<=4) and (today.strftime("%Y-%m-%d") not in HOLIDAY):
                pass
            elif today.strftime("%Y-%m-%d")in WORKDAY:
                pass
            else:
                time_start(9)
                continue

            item_list = []
            '''
            normal_tag
            '''
            file = base_file()
            for tag in tags:
                file['tenderName'] = tag
                r = requests.post(url, data=file)
                soup = BeautifulSoup(r.text, 'html5lib')

                try:
                    items = soup.find_all('tr', attrs={'onmouseover':'overcss(this);'})
                except:
                    time.sleep(3)
                    continue

                for item in items:
                    temp = get_list(item, tag)
                    item_list.append(temp)
                time.sleep(5)

            '''
            organ_tag
            '''
            file = base_file()
            for organ_tag in organ_tags:
                file['orgName'] = organ_tag
                r = requests.post(url, data=file)
                soup = BeautifulSoup(r.text, 'html5lib')

                try:
                    items = soup.find_all('tr', attrs={'onmouseover':'overcss(this);'})
                except:
    #                 time.sleep(3) #目前只有一例故取消延遲
                    continue

                for item in items:
                    temp = get_list(item, organ_tag)
                    item_list.append(temp)
    #             time.sleep(5) #目前只有一例故取消延遲

            df = pd.DataFrame(item_list, columns=['tag', '機關名稱', '標案案號', '標案名稱', '招標方式', '預算金額', '採購性質', '公告日期', '截止日期', 'url'])
            df.drop_duplicates('標案案號', inplace=True)

            data = pd.read_csv('/root/tenderdata/tender_list.csv')
            df = df[~df.標案案號.isin(list(data.標案案號))].reset_index(drop=True)
            if len(df)==0:
                Bot_Message('tenderlist', 'No new tender today.')

            elif len(df)==1:
                Bot_Message('tenderlist', '<{}|{}>\n預算：{}\n截止日期：{}'.format(df['url'][0], df['標案名稱'][0], df['預算金額'][0], df['截止日期'][0]))
                data = data.append(df).reset_index(drop=True)
                data.to_csv('tender_list.csv', index=False)
            else:
                message = []
                message_count = 0
                for i in df.index:
                    message.append('<{}|{}>\n預算：{}\n截止日期：{}'.format(df['url'][i], df['標案名稱'][i], df['預算金額'][i], df['截止日期'][i]))
                    message_count +=1
                    if message_count ==10:
                        Bot_Message('tenderlist', '\n\n'.join(message))
                        message = []
                        message_count = 0
                    else:
                        continue
                if message_count == 0:
                    pass
                else:
                    Bot_Message('tenderlist', '\n\n'.join(message))
                data = data.append(df).reset_index(drop=True)
                data.to_csv('/root/tenderdata/tender_list.csv', index=False)

            check_point = 0 #2 【偵錯器】
            del data, df

            if today.hour == 9:
                time_start(15)
            elif today.hour == 15:
                time_start(9)
            else:
                Bot_Message('tenderlist', '<@D.E.>\nTime ERROR in tender data!\nPlease check the code .')
                break

        except:
            check_point += 1 #2 【偵錯器】
            Bot_Message('tenderlist', '<@D.E.>\nGet ERROR in tender data!({}/{})\nPlease check the codes or website .'.format(check_point, 5))
            time.sleep(5)